In [1]:
# 通过openai包直接调用其他的大模型，例如此处的千问大模型
import os
from openai import OpenAI
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"), 
    # api_key='sk-25186dbe2f984cbcba48ffca2c81a78f',
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="qwen-plus", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}],
    )
    
print(completion.model_dump_json())

{"id":"chatcmpl-4057169e-6cf0-9167-8457-ba1a13c305b1","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"我是Qwen，由阿里云开发的超大规模语言模型。我被设计用于生成各种文本，如文章、故事、诗歌和代码等，并能根据不同的场景和需求进行对话。很高兴为您服务！如果您有任何问题或需要帮助，请随时告诉我。","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1730074340,"model":"qwen-plus","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":55,"prompt_tokens":32,"total_tokens":87,"completion_tokens_details":null,"prompt_tokens_details":null}}


In [2]:
# 通过langchain的langchain_openai包直接调用其他的大模型，例如此处的千问大模型
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=os.getenv("ZHIPU_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
#交流缓存记忆，将沟通的信息进行存储
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.invoke({"question": "tell me a joke"})


C:\Users\admin\AppData\Local\Temp\ipykernel_8476\548307798.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  conversation = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: tell me a joke

> Finished chain.


{'question': 'tell me a joke',
 'chat_history': [HumanMessage(content='tell me a joke'),
  AIMessage(content="Sure! Here's a lighthearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything! 🎉")],
 'text': "Sure! Here's a lighthearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything! 🎉"}

In [3]:
#MessagesPlaceholder的应用
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("history"),
    ("human", "{question}")
])

prompt.invoke({
    "history": [("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
    "question": "now multiply that by 4"
})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content="what's 5 + 2"), AIMessage(content='5 + 2 is 7'), HumanMessage(content='now multiply that by 4')])

In [37]:
#存储记忆的提取
memory.save_context({"input": "你好！"} ,{"outputs": "你好，有什么我可以帮助你的吗？"})
chat_history = memory.load_memory_variables({"input": "..."})
print(chat_history["chat_history"])

[HumanMessage(content='tell me a joke'), AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything! 😄"), HumanMessage(content='你好！'), AIMessage(content='你好，有什么我可以帮助你的吗？'), HumanMessage(content='你好！'), AIMessage(content='你好，有什么我可以帮助你的吗？')]


In [66]:

# 调用智谱大模型进行对话
from langchain_community.chat_models import ChatZhipuAI
zhipuai_chat = ChatZhipuAI(
        temperature=0.5,
        api_key=os.getenv("ZHIPU_API_KEY"),
        model="glm-4-air",
    )
zhipuai_chat.max_tokens = 5000 
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
messages = [
    # AIMessage(content="Hi."),
    SystemMessage(content="你是一位政府办公室的行政专员."),
    HumanMessage(content="撰写一篇“全面从严治党，永远在路上”为主题的报告，字数不少于1000字，不要初稿."),
]
# 打印结果
response = zhipuai_chat.invoke('撰写一篇“全面从严治党，永远在路上”为主题的报告，字数不少于1000字，不要初稿')
print(response.content)

ReadTimeout: The read operation timed out

In [53]:
# 通过agent 查询网站资料回答问题
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=5)]
prompt = hub.pull("hwchase17/react")
llm.max_tokens = 5000 
# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True,verbose=True)
agent_executor.invoke({"input": "撰写一篇“全面从严治党，永远在路上”为主题的报告，字数不少于1000字，不要初稿"})

c:\Users\admin\anaconda3\envs\qianwen\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
这个任务需要深入研究和理解全面从严治党的政策和实施情况。考虑到这是一篇较长的报告，我需要先梳理一下报告的大纲，然后逐部分进行撰写。

Action: tavily_search_results_json
Action Input: 全面从严治党 政策内容 实施情况 成效 挑战

Observation[{'url': 'http://www.qstheory.cn/qshyjx/2021-10/13/c_1127951594.htm', 'content': '深刻认识全面从严治党的重大意义\n全面从严治党，是党的十八大以来，以习近平同志为核心的党中央根据新的历史条件下党的建设面临的新情况和新问题，深刻总结历史上党通过自我革命保持先进性和纯洁性并领导各项事业胜利前进的宝贵经验，对管党治党做出的重大部署。它和全面建成小康社会（全面建设社会主义现代化国家）、全面依法治国、全面深化改革一起，构成了“四个全面”战略布局，在党的建设发展历史上、中国特色社会主义发展历史上具有重大意义。\n1.全面从严治党传承和弘扬了党的建设宝贵历史经验\n中国共产党是中国工人阶级先锋队，是中国人民和中华民族的先锋队。百年历史告诉我们，党的先进性和纯洁性，是党始终站在时代前列，始终和人民群众保持密切联系，能够经受住各种风浪考验的基本前提。正是因为具有先进性和纯洁性，中国共产党才能够在一百年的历史中不断发展壮大，不断走向成功。但是，保持先进性和纯洁性并不是一劳永逸的，中国共产党的先进性和纯洁性，是靠党根据环境和任务的变化，根据自身面临的新情况和新问题，不断清除自身存在的各种不健康因素，不断进行自我革命来实现的。毛泽东同志在民主革命时期就指出：“房子是应该经常打扫的，不打扫就会积满了灰尘；脸是应该经常洗的，不洗也就会灰尘满面。我们同志的思想，我们党的工作，也会沾染灰尘的，也应该打扫和洗涤。”中国共产党在历史上一贯重视从严管党治党，重视加强党内的思想教育、组织管理和纪律、作风要求。通过一次次的整党整风，一次次的党内思想教育，不断解决党内存在的思想、组织、作风等方面的问题。这种严格要求和从严治理，保证了党的团结统一，保证了党在新民主主义革命时期、社会主义革命和建设时期始终保持整体上的先进性和纯洁性，始终立于不败之地，并形成和

{'input': '撰写一篇“全面从严治党，永远在路上”为主题的报告，字数不少于1000字，不要初稿',
 'output': 'Agent stopped due to iteration limit or time limit.'}